In [0]:
from pyspark.sql.functions import *

In [0]:
df_policy_effect = spark.read.format("delta").load("/mnt/gold/policy_effect_on_covid")
df_mobility_effect = spark.read.format("delta").load("/mnt/gold/mobility_effects")

In [0]:
df_joined = df_policy_effect.join(
    df_mobility_effect,
    on=["location", "year", "month"],
    how="inner"
)

In [0]:
df_joined_2 = df_joined.select(
    "location", "year", "month",
    "avg_stringency_index", "monthly_policy_strictness_rank", "potential_policy_effective",
    "avg_workplace", "avg_home", "workplace_reduction_rank", "high_home_stay_flag", "mobility_effective_flag"
)

In [0]:
df_joined_3 = df_joined_2.withColumn(
    "combined_policy_mobility_effective",
    when(
        (col("potential_policy_effective") == 1) & (col("mobility_effective_flag") == 1), 1
    ).otherwise(0)
)

In [0]:
df_joined_3.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .save("/mnt/gold/combined_policy_mobility_effects")

In [0]:
df_joined_3.display()